In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import warnings

warnings.filterwarnings( 'ignore' )

In [3]:
train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv")
sample_result = pd.read_csv("sample_submission.csv")

In [4]:
x_data = train_data.loc[:, 'f0':'f99']
y_data = train_data.loc[:, 'loss']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test=train_test_split(x_data,
                                                  y_data,
                                                  test_size=0.2,   #전체 중 20%를 테스트용으로 분할
                                                                   #나머지 80%는 훈련용
                                                  shuffle=True,    #무작위로 섞어서 추출
                                                  random_state=20) #무작위 추출 시 일정한 기준으로

In [24]:
#  Decision tree의 수 (n_estimators) 
# - Strong law of large numbers 을 만족시키기 위해 2,000개 이상의 decision tree 필요 
#   (그러나, 아닌 경우도 많이  있음. search 및 최적화가 필요) (default 값이 대부분 10)
# Decision tree에서 노드 분할 시 무작위로 선택되는 변수의 수 (Max_features)
# - 일반적으로 변수의 수에 따라 다음과 같이 추천됨 (Diaz-Uriarte et el., 2006) (그러나, guide line일 뿐 search 필요)
# - 이 값이 크면 performance가 향상될 수 있지만, overfitting의 가능성 존재
# - Classification : sqrt(변수의 수)
# - Regression : 변수의 수/3
# - 옵션 : auto(제한 없음), sqrt(전체 feature 갯수에 square root), log2 (전체 feature 갯수에 log2)
# Max_depth 
# - 각 트리의 최대 depth를 의미함
# - Default 값은 대부분 None. 즉, 모든 leaf가 pure해질 때 까지 트리를 만드는 것. 
#  (Pure하다는 것은, 하나의 leaf에 해당하는 데이터가 동일한 class를 가지는 상태임)
# Min_sample_leaf 
# - 각 leaf 노드에서의 최소 데이터 갯수
# - Default 값은 대부분 1. 즉, 모든 leaf 노드는 1개 이상의 데이터를 가져야 함
# - 특히 regression 문제에서, smoothing 효과와 관련 있음
# - Training dataset의 갯수를 보고 결정해야 함
# Min_sample_split- Leaf 가 아닌 노드(internal node)에서의 최소 데이터 갯수
# - Default 값은 대부분 2. 즉, split 되기 전의 internal node는 2개 이상의 데이터를 가져야 함
# - 값이 너무 크면 underfitting 가능성 있음
# - Training dataset의 갯수를 보고 결정해야 함


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rnd_clf = RandomForestRegressor(n_estimators=2000, max_features=30, max_leaf_nodes=500, n_jobs=-1)
rnd_clf.fit(x_train, y_train)

RandomForestRegressor(max_features=30, max_leaf_nodes=500, n_estimators=2000,
                      n_jobs=-1)

In [25]:
y_pred = rnd_clf.predict(x_test)
rms = np.sqrt(mean_squared_error(y_test, y_pred))
print(rms)

7.850756646355922


In [19]:
# RandomForestRegressor(max_features=33, max_leaf_nodes=30, n_estimators=2000,
#                       n_jobs=-1)
# rms: 7.865402276518901
# score: 7.95618

# rnd_clf = RandomForestRegressor(n_estimators=5000, max_features=30, max_leaf_nodes=30, n_jobs=-1)
# rms: 7.865489629667556
# score: 

# rnd_clf = RandomForestRegressor(n_estimators=2000, max_features=30, max_leaf_nodes=50, n_jobs=-1)
# rms: 7.862920980137952
# score: 

# rnd_clf = RandomForestRegressor(n_estimators=2000, max_features=30, max_leaf_nodes=100, n_jobs=-1)
# rms: 7.859234815203407
# score: 

# rnd_clf = RandomForestRegressor(n_estimators=2000, max_features=30, max_leaf_nodes=500, n_jobs=-1)
# rms: 7.850756646355922
# score: 

In [26]:
new_x_test = test_data.loc[:, 'f0':'f99']
new_y_pred = rnd_clf.predict(new_x_test)


from pandas import Series, DataFrame

raw_data = {'id': [ i for i in range(250000,400000)],
            'loss': new_y_pred }

data = DataFrame(raw_data)
data.set_index('id', inplace=True)
print(data)
data.to_csv("submission.csv", mode='w')

            loss
id              
250000  6.739684
250001  6.336598
250002  6.258875
250003  6.691059
250004  7.330179
...          ...
399995  6.845035
399996  6.700499
399997  6.461808
399998  7.106361
399999  6.859786

[150000 rows x 1 columns]
